# Gestion des valeurs manquantes
La présence de valeurs manquantes exige le plus souvent d'être adressée car les modèles auxquels le *dataset* est soumis ne savent en général pas les gérer d'eux-même (XGBoost est un contre-exemple). 

*Remarque* : On s'intéresse ici aux valeurs manquantes parmi les *features*, dans le cas de où il nous manquerait des labels, d'autres stratégies s'appliquent comme l'apprentissage semi-supervisé.

## Stratégies 
On distingue deux attitudes complémentaires vis à vis des valeurs manquantes:
* **Imputer**: Chaque valeur manquante doit être remplacée par une valeur numérique dont l'estimation est plus ou moins sophitiquée suivant la méthode.
* **Ignorer**: Les observations présentant au moins une valeur manquante sont retirées du *dataset* pouvant conduire à une diminution inacceptable la taille du *dataset* (d'autant plus probable qu'on a beaucoup de variables). Attention, cette stratégie peut être le comportement adopté par défaut par un modèle ne supportant pas la présence de valeurs manquantes.

**Attention:** chacune des deux stratégies peut introduire un biais dans les données (ie. en modifier la distribution). Par exemple, imputer toutes les valeurs manquantes d'un champ par sa médiane fait piquer sa distribution au niveau de celle-ci. L'ampleur du biais introduit dépend principalement de:
* La méthode d'imputation,
* Du mécanisme ayant généré les valeurs manquantes (leur répartition),
* La proportion de valeurs manquantes: plus il y en a, moins on a d'information pour imputer et plus leur influence sur la distribution sera forte, 
* Plus largement, de l'information disponible dans le *dataset* pour imputer (si c'est la stratégie choisie).

*Remarque - Imputation*:
Les valeurs imputées le sont toujours avec une certaine variance. Or, le modèle auquel on passe un jeu de données aux valeurs manquantes imputées ne le sait pas et ses erreurs/variances et p-values seront donc ainsi sous-estimées (puisqu'elles ne tienne pas compte de cette source additionnelle de variance). Un paliatif est l'imputation multiple (*multiple imputation*) qui consiste à produire pour chaque valeur manquante non pas une mais plusieurs estimations. On obtient alors autant de *datasets* sur lesquels entrainer finalement notre modèle, l'ensemble devant permettre une estimation correcte des erreurs et des p-values associées. Voir aussi `sample_posterior=True` de `sklearn.impute.IterativeImputer`.

## Génération des valeurs manquantes
La caractérisation du caractère aléatoire ou non de la répartition des valeurs manquantes est déterminant dans le choix de l'attitude à adopter. On distingue du plus bénin au plus difficile: 
* ***Missing Completely at Random*** : L'absence de valeur est totalement aléatoire, ne dépend pas de la valeur prise par des variables observées (ou inobservées), ni de la variable elle même. C'est le cas le moins défavorable dans le sens où ignorer les observations incomplètes n'introduit pas de biais (on dit que ces valeurs manquantes sont "ignorables"). Inversement, imputer sur la portion complète du *dataset* ne résultera pas en un estimateur biaisé.
* ***Missing at Random***: L'absence de valeur n'est pas totalement aléatoire mais dépend d'une ou plusieurs variables observables (ie. qu'on a à disposition). Ignorer nous expose à l'introduction d'un biais. Imputer en revanche ne pose qu'un problème limité pourvu que le modèle utilisé intègre les variables permettant d'expliquer l'absence de valeur (et dont on suppose qu'elles ne présentes pas elles mêmes de valeurs manquantes). Ex: Dans un sondage recueillant notamment le sexe des sondés, la propension à refuser de donner son poids peut être plus élevée chez les femmes.
* ***Missing at Random* avec dépendance sur une ou plusieurs variables inobservées**: Le caractère manquant ou non n'est pas totalement aléatoire mais dépend totalement ou en partie de variables auxquelles on a pas accès. Comme ci-dessus, ignorer nous expose à l'introduction d'un biais mais imputer est plus difficile. Ex: Les patients chez qui un traitement en cours de test suscite un inconfort on une propension plus élevée à ne pas aller au bout du traitement et on peut ne pas avoir de mesure de cet inconfort.
* **Absence de valeur dépendant de la variable elle même**: Il existe une relation entre les valeurs prises par la variable et la propension à l'absence de valeur. C'est le cas le plus défavorable au sens où ignorer les valeurs manquantes (qui sont alors dites *non ignorable*) peut conduire aux biais les plus importants et où l'imputation peut être difficile. Le mécanisme générant l'absence de valeur doit alors être modélisé ou alors on essaye au minimum de se rapprocher du cas *Missing at Random* en simulant la variable à l'aide d'autres prédicteurs. Ex: Si on interroge quelqu'un sur sa consommation de cocaïne, l'absence se valeur risque d'être d'autant plus élevée que la consommation est élevée.

*Remarque - Tester MCAR vs MAR*:
Une façon de procéder peut être de créer pour une variable aux valeurs manquante donnée une *dummy variable* (1=*missing* et 0=*observed*) et d'effectuer des t-tests/tests du khi-deux entre cette variable et les autres. Ex: Si la variable où les valeurs manquent est le poids et qu'on teste contre le sexe (variable catégorielle), un khi-deux va nous dire si la proportion de valeurs manquantes (moyenne de la *dummy variable* sur la catégorie) dépend où non du sexe (correspond à une ANOVA).

## `sklearn.impute`
La librairie met à disposition deux types d'objets intégrables à une pipeline et permettant d'imputer des valeurs manquantes:
* `sklearn.impute.SimpleImputer`: Donne accès aux stratégies d'imputations basiques: remplacement des valeurs manqauntes par la moyenne, médiane (plus robuste que la moyenne pour les *features* à queue épaisse dont quelques valeurs risquent d'avoir une influence élevée sur le résultat), valeur la plus fréquente ou une constante.
* `sklearn.impute.IterativeInputer`: Permet de modéliser la variable aux valeurs manquantes à l'aide des autres variables (ie. à partir de l'information disponible dans le *dataset*). On utilise simplement des modèles classiques d'apprentissage supervisé: régression linéaire, *random forest*, kNN, etc. La procédure consiste en:
    * Une étape d'initialisation à l'aide d'une première imputation simple pour chacune des variables présentant des valeurs manquantes à l'aide d'un `SimpleImputer`. Le masque des valeurs manquantes est conservé (c'est l'objet de `sklearn.impute.MissingIndicator`).
    * La répétition d'une étape où pour chacune des variables présentant des valeurs manquantes (l'ordre dans lequel elles sont choisies est contrôlé par `imputation_strategy`), on va entrainer un modèle (argument `estimator`) sur les autres variables (tout ou partie, on doit dans ce dernier cas définir une stratégie avec `n_nearest_features`) et l'utiliser pour prédire les valeurs manquantes de la variable courante. Les valeurs calculées remplacent les valeurs imputées à l'étape précédente. L'imputation s'arrête soit lorsque les valeurs imputées se stabilisent pour une tolérance `tol` donnée ou après un nombre maximum d'itérations `max_iter` donné.

### Voir aussi
* http://www.stat.columbia.edu/~gelman/arm/missing.pdf
* https://hal.archives-ouvertes.fr/hal-02024202/file/main.pdf